In [3]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over SUPPORT


In [4]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [5]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

FileNotFoundError: [Errno 2] No such file or directory: 'statistics/deep_hit/best_model.pkl'

## `c index`

In [ ]:
c_index_df

In [ ]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, val)

___

## `concordance td`

In [6]:
concordance_td_df

NameError: name 'concordance_td_df' is not defined

In [13]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, val)

deep_surv 0.6716753022452504
reg_coxph 0.6639032815198618
rsf 0.6589810017271157
cox_time 0.6576856649395509
deep_hit 0.6037996545768567
pc_hazard 0.49758203799654577


## `integrated brier score`

In [14]:
ibs_df

,ibs,ibs_std,ibs_params
deep_hit,0.113506,0.001865,"deep_hit_dropout_0.1_num_nodes_[16, 32, 64, 64..."
deep_surv,0.166265,0.006674,deep_surv_dropout_0.16_num_nodes_[41]_activati...
cox_time,0.166580,0.006295,"cox_time_dropout_0.1_num_nodes_[32, 32]_activa..."
rsf,0.168998,0.004748,rsf_n_estimators_200_max_depth_5_min_samples_s...
reg_coxph,0.170414,0.007362,reg_coxph_l1_ratio_0.1_tol_0.1_max_iter_1000_v...
pc_hazard,0.247230,0.008502,"pc_hazard_dropout_0.3_num_nodes_[16, 16, 16, 1..."


In [15]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, val)

reg_coxph 0.13610432931664593
deep_surv 0.14188171823718218
cox_time 0.1479041763159939
rsf 0.1513038330276852
deep_hit 0.3381021312429789
pc_hazard 0.44771241187895056
